In [ ]:
from tweepy import API
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from Tweepy_API import config

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Authenticator
class TwitterAuthenticator():
    def authenticate_twitter(self):
        auth = OAuthHandler(config.API_KEY, config.API_SECRET_KEY)
        auth.set_access_token(config.ACCESS_TOKEN, config.ACCESS_TOKEN_SECRET)
        return auth     


# Streamer
class TwitterStreamer():
    """
    class for streaming and processing live tweets
    """
    def __init__(self):
        self.twitter_auth = TwitterAuthenticator()
        
    def stream_tweets(self, hashtags_list):
        # Handles twitter authentication and connection to twitter streamer API
        listener = TwitterListener()
        auth = self.twitter_auth.authenticate_twitter()
        stream = Stream(auth, listener)
        stream.filter(track=hashtags_list, is_async=True)
        

# Streamer Listener
class TwitterListener(StreamListener):
    """
    Basic listener class that prints to cmd
    """
    def on_data(self, data):
        print(data)
        return True
    
    def on_error(self, status):
        # Return False in case rate limit occurs
        if status==402:
            return False
        print(status)

In [ ]:
# Client
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter()
        self.twitter_client = API(self.auth)
        self.twitter_user  =twitter_user
    
    def get_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets
    
    def get_frds(self, num_frds):
        frds = []
        for tweet in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_frds):
            frds.append(tweet)
        return frds

In [ ]:
class TweetAnalysizer():
    
    def tweets2df(self, tweets):
        df = pd.DataFrame(data = [tweet.text for tweet in tweets], columns=["Tweets"])
        df["id"] = np.array([tweet.id for tweet in tweets])
        df["geo"] = np.array([tweet.geo for tweet in tweets])
        df["date"] = np.array([tweet.created_at for tweet in tweets])
        df["likes"] = np.array([tweet.favorite_count for tweet in tweets])
        
        return df

In [ ]:
# hashtags_list = ["nvidia"]
# twitter_streamer = TwitterStreamer()
# tweets = twitter_streamer.stream_tweets(hashtags_list)

twitter_client = TwitterClient("realdonaldtrump")
tweets = twitter_client.get_tweets(10)

df = TweetAnalysizer().tweets2df(tweets)
df
# Time Series
time_likes = pd.Series(data=df["likes"].values, index=df["date"])
time_likes.plot(color="b")
plt.show()

In [ ]:
query="#nvidia"

auth = TwitterAuthenticator().authenticate_twitter()
twitter_api = API(auth)


tweets = Cursor(twitter_api.search, q=query, lang="en").items(200)
for i,tweet in enumerate(tweets):
    print(i,tweet.text)
    print()

In [ ]:
tweets = twitter_api.search(q=query, lang="en",count=150)
for i,tweet in enumerate(tweets):
    print(i,tweet.text)
    print()

In [6]:
from Tweepy_API.TwitterClient import TwitterClient

client = TwitterClient("nvidia").getTweets(5)
df = client.getTweetDataFrame()

df

,Id,Date,Tweet,Geo,Likes,Retweets
0,1301203944593649667,2020-09-02 17:02:50,RT @NVIDIADRIVE: Watch NVIDIA founder and CEO ...,None,0,43
1,1301182835546439684,2020-09-02 15:38:57,RT @NVIDIACreators: Turn any room into a home ...,None,0,95
2,1301171539488858113,2020-09-02 14:54:04,RT @NVIDIADRIVE: With the next-gen MBUX #AI co...,None,0,78
3,1300863197440663552,2020-09-01 18:28:49,RT @NVIDIAGeForce: New RT Cores. \nNew Tensor ...,None,0,10547
4,1300860795723669505,2020-09-01 18:19:17,"GeForce RTX 30 Series GPUs, powered by our sec...",None,833,170
